In [ ]:
# %%bash
# rm -rf /kaggle/working
# ln -s /kaggle/input/palmbigdatabase/PalmBigDataBase /kaggle/working

rm: cannot remove '/kaggle/working': Device or resource busy


In [ ]:
import torch
import torch.nn as nn
import yaml

with open("args.yml", "r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)

print(cfg["train"]["batch_size"])

32


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 53 * 53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 53 * 53)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [2]:
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T

class PalmDataset(Dataset):
    def __init__(self, root, transform=None, target_transform=None):
        self.root = Path(root)
        self.samples = sorted(self.root.glob("*.bmp"))
        self.transform = transform
        self.target_transform = target_transform

        ids = sorted({self._get_identity(p.name) for p in self.samples})
        self.id2idx = {pid: idx for idx, pid in enumerate(ids)}

    @staticmethod
    def _get_identity(filename):
        # 例：P_F_100_1.bmp → 100
        return int(filename.split("_")[2])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        path = self.samples[index]
        with Image.open(path) as img:
            img = img.convert("L")  # 灰度

        label = self.id2idx[self._get_identity(path.name)]
        if self.transform:
            img = self.transform(img)
        if self.target_transform:
            label = self.target_transform(label)

        return img, label

# 预处理与 DataLoader
transform = T.Compose([
    T.Resize((cfg["img_basic_info"]["img_height"], cfg["img_basic_info"]["img_width"])),
    T.ToTensor(),
    T.Normalize(mean=[0.5], std=[0.5]),
])

dataset = PalmDataset("PalmBigDataBase", transform=transform)

train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len
train_set, val_set = random_split(dataset, [train_len, val_len])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False, num_workers=4)


In [ ]:
def fit():
    model = Net()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(10):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to("cpu"), labels.to("cpu")

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    print("Finished Training")

fit()

In [16]:
from PIL import ImageStat

with Image.open("PalmBigDataBase/P_F_100_1.bmp") as img:
    print("Format:", img.format)
    print("Mode:", img.mode)
    print("Size (W,H):", img.size)
    print("Info dict:", img.info)
    print("Bands:", img.getbands())
    print("Extrema (min,max) per band:", img.getextrema())

    stat = ImageStat.Stat(img)
    print("Mean per band:", stat.mean)
    print("RMS per band:", stat.rms)

    w, h = img.size
    cx, cy = w // 2, h // 2
    print("Center pixel (x,y)={} value: {}".format((cx, cy), img.getpixel((cx, cy))))

    hist = img.histogram()
    print("Histogram length:", len(hist))
    print("First 10 histogram values:", hist[:10])

Format: BMP
Mode: L
Size (W,H): (128, 128)
Info dict: {'dpi': (0.0, 0.0), 'compression': 0}
Bands: ('L',)
Extrema (min,max) per band: (0, 255)
Mean per band: [138.5712890625]
RMS per band: [142.6283460353286]
Center pixel (x,y)=(64, 64) value: 133
Histogram length: 256
First 10 histogram values: [1, 0, 0, 0, 1, 0, 1, 2, 0, 0]
